**1단계 : 케라스모델파일 업로드**

In [1]:
# 사용자 컴퓨터에서 keras_model.h5 파일을 Colab 서버 컴퓨터로 업로드
from google.colab import files
print('Upload keras_model.h5')
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn]))) 
!ls

Upload keras_model.h5


Saving keras_model.h5 to keras_model.h5
User uploaded file "keras_model.h5" with length 2453432 bytes
keras_model.h5	sample_data


**2단계 : frozen model로 변환**

In [2]:
# keras_model.h5를 frozen_model.pb 파일로 변환 
import tensorflow as tf
import os
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

#loaded = tf.saved_model.load('/content')          # tf.saved_model.save로 저장한것을 로드할때
loaded = tf.keras.models.load_model('/content/keras_model.h5')  # tf.keras.models.save_model로 저장한것을 로드할때

print(loaded.inputs[0].shape)
print(loaded.inputs[0].dtype)

# define the directory for .pb model
pb_model_path = '/content'
# define the name of .pb model
pb_model_name = 'frozen_model.pb'
# create directory for further converted model
os.makedirs(pb_model_path, exist_ok=True)
# get model TF graph
tf_model_graph = tf.function(lambda x: loaded(x))
# get concrete function
tf_model_graph = tf_model_graph.get_concrete_function(tf.TensorSpec(loaded.inputs[0].shape, loaded.inputs[0].dtype))
# obtain frozen concrete function
frozen_tf_func = convert_variables_to_constants_v2(tf_model_graph)
# get frozen graph
graph_def = frozen_tf_func.graph.as_graph_def()
tf.io.write_graph(graph_or_graph_def=graph_def, logdir=pb_model_path, name=pb_model_name, as_text=False)

(None, 224, 224, 3)
<dtype: 'float32'>


'/content/frozen_model.pb'

**3단계 : frozen model파일 다운로드**

In [3]:
# Colab 서버 컴퓨터에서 사용자 컴퓨터로 frozen_model.pb 파일을 다운로드
from google.colab import files
print('Download frozen_model.pb')
uploaded = files.download('/content/frozen_model.pb')

Download frozen_model.pb


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
"""
# python opencv example
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

#net = cv.dnn.readNetFromTensorflow('/content/frozen_model.pb')
net = cv.dnn.readNet('/content/frozen_model.pb')
# read the image
#input_img = cv.imread("/content/O_15.jpg", cv.IMREAD_COLOR)
input_img = cv.imread("/content/X_15.jpg", cv.IMREAD_COLOR)
plt.imshow(input_img)
plt.show()
#input_img = input_img.astype(np.float32)
scale = 1 / 255.0
# prepare input blob to fit the model input:
# 1. subtract mean
# 2. scale to set pixel values from 0 to 1
input_blob = cv.dnn.blobFromImage(
    image=input_img,
    scalefactor=scale,
    size=(224, 224),  # img target size
#    swapRB=True,  # BGR -> RGB
#    crop=True  # center crop
)
print("Input blob shape: {}\n".format(input_blob.shape))
# set OpenCV DNN input
net.setInput(input_blob)
# OpenCV DNN inference
out = net.forward()
print("OpenCV DNN prediction: \n")
print("* shape: ", out.shape)
# get the predicted class ID
imagenet_class_id = np.argmax(out)
# get confidence
class_names = ['O','X']
confidence = out[0][imagenet_class_id]
print("* class ID: {}, label: {}".format(imagenet_class_id, class_names[imagenet_class_id]))
print("* confidence: {:.4f}\n".format(confidence))
"""